In [ ]:
import numpy as np
import random
import copy
import os
import time
import matplotlib.pyplot as plt

In [ ]:
# =============================================================================
# --- CÉLULA 0: MONTAR O GOOGLE DRIVE ---
# =============================================================================
from google.colab import drive
drive.mount('/content/drive')

# Cria a pasta de treino no seu Drive, se ela não existir
import os
SAVE_DIR = "/content/drive/MyDrive/OthelloTreinoTD0"
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Arquivos de treino serão salvos e carregados de: {SAVE_DIR}")

In [ ]:
class Othello:
    direcoes_captura = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]

    def __init__(self, outro=None):
        if outro is not None:
            self._cols = outro._cols
            self._lins = outro._lins
            self._jogador_atual = outro._jogador_atual
            self._tab = np.array(outro._tab, dtype=np.int8)
            self._terminou = outro._terminou
            self._placar_cache = tuple(outro._placar_cache)
        else:
            self._cols = 8
            self._lins = 8
            self._tab = np.zeros((8,8), dtype=np.int8)
            self._tab[3,3] = -1
            self._tab[4,4] = -1
            self._tab[3,4] = 1
            self._tab[4,3] = 1
            self._jogador_atual = 1
            self._terminou = False
            self._placar_cache = self._calc_placar()
        self._jogadas_cache = None
        self._capturas_cache = None

    def dim(self):
        return (self._cols, self._lins)

    def tabuleiro(self):
        return np.array(self._tab, copy=True)

    def posicao(self, coord):
        return self._tab[coord[0], coord[1]]

    def jogador_atual(self):
        return int(self._jogador_atual)

    def _calc_placar(self):
        pretas = int(np.sum(self._tab == 1))
        brancas = int(np.sum(self._tab == -1))
        return (pretas, brancas)

    def placar(self):
        self._placar_cache = self._calc_placar()
        return self._placar_cache

    def _lista_de_capturas_a_partir(self, i, j, jogador):
        if self._tab[i,j] != 0:
            return []
        capturas_total = []
        for di, dj in Othello.direcoes_captura:
            ci, cj = i+di, j+dj
            caps_dir = []
            while 0 <= ci < 8 and 0 <= cj < 8 and self._tab[ci,cj] == -jogador:
                caps_dir.append((ci,cj))
                ci, cj = ci+di, cj+dj
            if 0 <= ci < 8 and 0 <= cj < 8 and self._tab[ci,cj] == jogador and len(caps_dir) > 0:
                capturas_total.extend(caps_dir)
        return capturas_total

    def jogadas_legais(self):
        if self._terminou:
            return []
        if self._jogadas_cache is not None:
            return list(self._jogadas_cache)
        jogador = self._jogador_atual
        jogadas = []
        capt = {}
        for i in range(8):
            for j in range(8):
                caps = self._lista_de_capturas_a_partir(i,j,jogador)
                if len(caps) > 0:
                    jogadas.append((i,j))
                    capt[(i,j)] = caps
        self._jogadas_cache = jogadas
        self._capturas_cache = capt
        return list(jogadas)

    def _aplica_jogada_inplace(self, jogada):
        i, j = jogada
        if self._capturas_cache is None:
            caps = self._lista_de_capturas_a_partir(i, j, self._jogador_atual)
        else:
            caps = self._capturas_cache.get(jogada, [])

        self._tab[i,j] = self._jogador_atual
        for (ci,cj) in caps:
            self._tab[ci,cj] = self._jogador_atual

        self._jogadas_cache = None
        self._capturas_cache = None

        self._jogador_atual = -self._jogador_atual

        if len(self.jogadas_legais()) == 0:
            self._jogador_atual = -self._jogador_atual
            self._jogadas_cache = None
            self._capturas_cache = None
            if len(self.jogadas_legais()) == 0:
                self._terminou = True


    def joga(self, jogada):
        if self._terminou:
            raise RuntimeError("Jogo encerrado")
        if jogada not in self.jogadas_legais():
            raise RuntimeError("Jogada ilegal")
        novo = Othello(self)
        novo._aplica_jogada_inplace(jogada)
        return novo

    def terminou(self):
        return bool(self._terminou)

print("Célula 1 (Motor Othello) carregada.")

In [ ]:
# =============================================================================
# --- CÉLULA 2: DADOS DA TABELA E CLASSES DOS JOGADORES (CORRIGIDO PARA ESTABILIDADE) ---
# =============================================================================

indice_coeficientes = [
  [  0,  1,  2,  3,  3,  2,  1,  0],
  [  1,  4,  5,  6,  6,  5,  4,  1],
  [  2,  5,  7,  8,  8,  7,  5,  2],
  [  3,  6,  8,  9,  9,  8,  6,  3],
  [  3,  6,  8,  9,  9,  8,  6,  3],
  [  2,  5,  7,  8,  8,  7,  5,  2],
  [  1,  4,  5,  6,  6,  5,  4,  1],
  [  0,  1,  2,  3,  3,  2,  1,  0]]

max_coef = 9
coordenadas_com_coeficiente = [[] for i in range(max_coef+1)]
for i, linha in enumerate(indice_coeficientes):
  for j, idx in enumerate(linha):
    coordenadas_com_coeficiente[idx].append((i,j))
# --- Fim dos dados da tabela ---


# --- Jogador Aleatório (para avaliação) ---
class JogadorAleatorio:
    def __init__(self, seed=None):
        self.rng = np.random.default_rng(seed)
    def nova_partida(self, jogo, jogador, id_oponente=None):
        pass
    def escolhe_jogada(self, jogo: Othello):
        jogs = jogo.jogadas_legais()
        if not jogs:
            raise RuntimeError("Sem jogadas legais")
        return jogs[int(self.rng.integers(0, len(jogs)))]
    def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois): pass
    def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois): pass
    def informa_fim(self, jogo_final): pass


# --- Jogador Posicional (Heurístico, do seu notebook VNet) ---
class JogadorPosicional:
    def __init__(self, w=None):
        # Pesos heurísticos
        self.w = {
            "corner":   100.0,
            "x_pen":     80.0,
            "c_pen":     40.0,
            "mobility":   5.0,
            "edge":       2.0,
            "disc":       0.2
        }
        if w:
            self.w.update(w)
        self._sou = +1

        self.corners = [(0,0),(0,7),(7,0),(7,7)]
        self.x_of = { (0,0):(1,1), (0,7):(1,6), (7,0):(6,1), (7,7):(6,6) }
        self.c_of = {
            (0,0): [(0,1),(1,0)],
            (0,7): [(0,6),(1,7)],
            (7,0): [(6,0),(7,1)],
            (7,7): [(6,7),(7,6)]
        }

    def nova_partida(self, jogo, jogador, id_oponente=None):
        self._sou = int(jogador)

    def escolhe_jogada(self, jogo):
        jogs = jogo.jogadas_legais()
        if not jogs:
            raise RuntimeError("Sem jogadas legais")
        melhor, best = None, -1e18
        for a in jogs:
            sp = jogo.joga(a)
            val = self._heuristic(sp, self._sou)
            if val > best:
                best, melhor = val, a
        return melhor

    def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois): pass
    def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois): pass
    def informa_fim(self, jogo_final): pass

    def _heuristic(self, jogo, perspective):
        tab = jogo.tabuleiro()
        w = self.w

        # 1) cantos
        corner_score = 0
        for (i,j) in self.corners:
            if tab[i,j] == perspective: corner_score += 1
            elif tab[i,j] == -perspective: corner_score -= 1

        # 2) X/C penalties
        x_pen, c_pen = 0, 0
        for c in self.corners:
            ci, cj = c
            if tab[ci,cj] == 0:
                xi,xj = self.x_of[c]
                if tab[xi,xj] == perspective: x_pen += 1
                for (pi,pj) in self.c_of[c]:
                    if tab[pi,pj] == perspective: c_pen += 1

        # 3) mobilidade
        mob_self  = self._count_legal_moves_for(jogo, perspective)
        mob_opp   = self._count_legal_moves_for(jogo, -perspective)
        mobility  = mob_self - mob_opp

        # 4) bordas
        edge_self = edge_opp = 0
        for k in range(8):
            edge_self  += (tab[0,k] == perspective) + (tab[7,k] == perspective)
            edge_opp   += (tab[0,k] == -perspective) + (tab[7,k] == -perspective)
            edge_self  += (tab[k,0] == perspective) + (tab[k,7] == perspective)
            edge_opp   += (tab[k,0] == -perspective) + (tab[k,7] == -perspective)
        edge_score = edge_self - edge_opp

        # 5) contagem de peças
        disc_score = int(np.sum(tab == perspective)) - int(np.sum(tab == -perspective))

        score = (
            w["corner"] * corner_score
            - w["x_pen"] * x_pen
            - w["c_pen"] * c_pen
            + w["mobility"] * mobility
            + w["edge"] * edge_score
            + w["disc"] * disc_score
        )
        return float(score)

    def _count_legal_moves_for(self, jogo, who):
        # Othello(jogo) usa o construtor de cópia
        g = Othello(jogo)
        g._jogador_atual = who
        g._jogadas_cache = None
        g._capturas_cache = None
        return len(g.jogadas_legais())
# --- Fim do Jogador Posicional ---


# --- JOGADOR DE TABELA (TREINÁVEL COM TD-0 ESTÁVEL) ---
class JogadorTabelaPosicoes():
  def __init__(self, indice_coeficientes,
               tabela_primeiro_jogador, tabela_segundo_jogador,
               alpha=0.001, epsilon=0.1, gamma=1.0, is_training=False):

    self._lins = len(indice_coeficientes)
    self._cols = len(indice_coeficientes[0])
    self._idxcoef = indice_coeficientes

    self._tabela_1 = tabela_primeiro_jogador
    self._tabela_2 = tabela_segundo_jogador

    self._alpha = alpha
    self._epsilon = epsilon
    self._gamma = gamma
    self._is_training = is_training

    self._last_state_features = None
    self._jogador = 0
    self._minha_tabela = None
    self._tabela_oponente = None


  def nova_partida(self, jogo, jogador, id_oponente = None):
    self._jogador = jogador

    if self._jogador == 1:
        self._minha_tabela = self._tabela_1
        self._tabela_oponente = self._tabela_2
    else:
        self._minha_tabela = self._tabela_2
        self._tabela_oponente = self._tabela_1

    if self._is_training:
      self._last_state_features = None

  # --- FUNÇÕES DE CÁLCULO ---

  def _get_features(self, jogo, player_perspective):
    features = [0.0] * (max_coef + 1)
    for i in range(self._lins):
      for j in range(self._cols):
        idx_peso = self._idxcoef[i][j]
        features[idx_peso] += jogo.posicao((i,j)) * player_perspective
    return features

  def _valor_linear(self, features_estado, table_to_use):
    val = 0.0
    for i in range(len(table_to_use)):
      val += table_to_use[i] * features_estado[i]
    return val

  def _get_squashed_value(self, jogo_estado, player_perspective, table_to_use):
    features = self._get_features(jogo_estado, player_perspective)
    val_linear = self._valor_linear(features, table_to_use)

    return 2 / (1 + np.exp(-2 * val_linear)) - 1


  def escolhe_jogada(self, jogo):
    jogadas_possiveis = jogo.jogadas_legais()
    if not jogadas_possiveis:
        raise RuntimeError("Sem jogadas legais")

    if self._is_training and random.random() < self._epsilon:
      return random.choice(jogadas_possiveis)

    valores = []
    for jogada in jogadas_possiveis:
        proximo_estado = jogo.joga(jogada)

        v_oponente = self._get_squashed_value(proximo_estado,
                                              -self._jogador,
                                              self._tabela_oponente)

        valores.append(-v_oponente)

    return jogadas_possiveis[np.argmax(valores)]


  # --- APRENDIZADO TD(0) - ATUALIZAÇÃO NO MEIO DO JOGO ---
  def informa_propria_jogada(self, tabuleiro_antes, jogada, tabuleiro_depois):
    if not self._is_training:
      return

    # 1. Pega os features do estado ATUAL (s_t) da *minha* perspectiva
    features_st = self._get_features(tabuleiro_antes, self._jogador)

    # 2. Calcula o valor de V(s_{t+1}) usando a tabela do *oponente* (esmagado)
    v_st_plus_1_predito = self._get_squashed_value(tabuleiro_depois,
                                                   -self._jogador,
                                                   self._tabela_oponente)

    # 3. O Alvo TD é (R + gamma * -V(s_{t+1}))
    target_td = 0.0 + self._gamma * (-v_st_plus_1_predito)

    # 4. Pega a predição de V(s_t) usando a *minha* tabela (esmagado)
    v_st_predito = self._get_squashed_value(tabuleiro_antes,
                                            self._jogador,
                                            self._minha_tabela)

    # 5. Calcula o Erro (agora é uma subtração de números pequenos)
    erro = target_td - v_st_predito

    # 6. Atualiza os pesos da *minha* tabela para o estado s_t
    for i in range(len(self._minha_tabela)):
      self._minha_tabela[i] += self._alpha * erro * features_st[i]


  def informa_jogada_oponente(self, tabuleiro_antes, jogada, tabuleiro_depois):
    if self._is_training:
        self._last_state_features = self._get_features(tabuleiro_depois, self._jogador)


  # --- APRENDIZADO TD(0) - ATUALIZAÇÃO FINAL ---
  def informa_fim(self, jogo_final):
    if not self._is_training:
      return

    if jogo_final.jogador_atual() == self._jogador:
        self._last_state_features = self._get_features(jogo_final, self._jogador)

    if self._last_state_features is None:
       return

    # 1. Pega o resultado real (o "target" final)
    p1, p2 = jogo_final.placar()
    if p1 > p2: winner = 1.0
    elif p2 > p1: winner = -1.0
    else: winner = 0.0

    target_real = winner * self._jogador

    # 2. Pega a predição (esmagada) do *último* estado
    v_last_predito = self._get_squashed_value(jogo_final,
                                              self._jogador,
                                              self._minha_tabela)

    # 3. Calcula o Erro final (Alvo Real - Última Predição)
    erro_final = target_real - v_last_predito

    # 4. Atualiza os pesos da *minha* tabela uma última vez
    for i in range(len(self._minha_tabela)):
      self._minha_tabela[i] += self._alpha * erro_final * self._last_state_features[i]

print("Célula 2 (Jogadores TD-0 Estável e Posicional) carregada.")

In [ ]:
# =============================================================================
# --- CÉLULA 3: FUNÇÕES DE PARTIDA E AVALIAÇÃO (ADD POSICIONAL) ---
# =============================================================================

def play_game(j1, j2, return_states=False):
    g = Othello()
    j1.nova_partida(g, +1)
    j2.nova_partida(g, -1)
    states = []
    while not g.terminou():
        if return_states:
            states.append(Othello(g))

        g_antes = g
        if g.jogador_atual() == +1:
            a = j1.escolhe_jogada(g)
        else:
            a = j2.escolhe_jogada(g)

        g2 = g.joga(a)

        if g.jogador_atual() == +1:
            j1.informa_propria_jogada(g_antes, a, g2)
            j2.informa_jogada_oponente(g_antes, a, g2)
        else:
            j2.informa_propria_jogada(g_antes, a, g2)
            j1.informa_jogada_oponente(g_antes, a, g2)
        g = g2

    p, b = g.placar()
    winner = +1 if p > b else (-1 if b > p else 0)

    j1.informa_fim(g)
    j2.informa_fim(g)

    if return_states:
        return winner, states
    return winner

def evaluate_vs_random(current_weights_p1, current_weights_p2, games=100):

    jogador_tabela_eval = JogadorTabelaPosicoes(
        indice_coeficientes,
        current_weights_p1,
        current_weights_p2,
        is_training=False,
        epsilon=0.0
    )

    jogador_aleatorio = JogadorAleatorio()

    wins = 0
    losses = 0
    draws = 0

    for k in range(games):
        if k % 2 == 0:
            winner = play_game(jogador_tabela_eval, jogador_aleatorio)
        else:
            winner = play_game(jogador_aleatorio, jogador_tabela_eval)
            winner = -winner

        if winner > 0: wins += 1
        elif winner < 0: losses += 1
        else: draws += 1

    winrate = wins / games
    return {"winrate": winrate, "wins": wins, "losses": losses, "draws": draws}


def evaluate_vs_positional(current_weights_p1, current_weights_p2, games=100):

    jogador_tabela_eval = JogadorTabelaPosicoes(
        indice_coeficientes,
        current_weights_p1,
        current_weights_p2,
        is_training=False,
        epsilon=0.0
    )

    jogador_posicional = JogadorPosicional()

    wins = 0
    losses = 0
    draws = 0

    for k in range(games):
        if k % 2 == 0:
            # Joga como P1
            winner = play_game(jogador_tabela_eval, jogador_posicional)
        else:
            # Joga como P2
            winner = play_game(jogador_posicional, jogador_tabela_eval)
            winner = -winner

        if winner > 0: wins += 1
        elif winner < 0: losses += 1
        else: draws += 1

    winrate = wins / games
    return {"winrate": winrate, "wins": wins, "losses": losses, "draws": draws}


print("Célula 3 (Funções de Jogo e Avaliação) carregada.")

In [ ]:
# =============================================================================
# --- CÉLULA 4: LOOP DE TREINAMENTO (COM POPULAÇÃO DE OPONENTES) ---
# =============================================================================

try:
    Othello
    JogadorPosicional
    evaluate_vs_random
except NameError:
    print("ERRO: Por favor, rode as Células 1, 2 e 3 primeiro!")
    raise

import os
import time
import matplotlib.pyplot as plt
import csv
import pandas as pd
import random

try:
    from google.colab import drive
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

if __name__ == "__main__":

    # --- Hiperparâmetros ---
    NUM_PARTIDAS_TREINO = 2000000
    ALPHA = 0.001
    EPSILON_TREINO = 0.1
    GAMMA_TREINO = 1.0
    AVALIAR_RANDOM_A_CADA = 10000
    AVALIAR_POS_A_CADA_N_JOGOS = 50000
    JOGOS_AVAL_RANDOM = 200
    JOGOS_AVAL_POS = 50

    # --- Caminhos dos Arquivos ---
    if IS_COLAB:
        try:
            print("Montando Google Drive...")
            drive.mount('/content/drive')
            SAVE_DIR = "/content/drive/MyDrive/OthelloTreinoTD0"
        except Exception as e:
            print(f"Falha ao montar Drive, salvando localmente. Erro: {e}")
            SAVE_DIR = "OthelloTreinoTD0_local"
    else:
        SAVE_DIR = "OthelloTreinoTD0"

    os.makedirs(SAVE_DIR, exist_ok=True)

    FILE_PESOS_P1 = os.path.join(SAVE_DIR, "tabela_linear_p1_td0.npy")
    FILE_PESOS_P2 = os.path.join(SAVE_DIR, "tabela_linear_p2_td0.npy")
    LOG_FILE = os.path.join(SAVE_DIR, "treino_log.csv")
    PLOT_FILE = os.path.join(SAVE_DIR, "progresso_treino_tabela_td0.png")
    PESOS_LOG_FILE = os.path.join(SAVE_DIR, "treino_log_pesos.csv")
    PLOT_PESOS_FILE = os.path.join(SAVE_DIR, "progresso_pesos_p1.png")

    # --- Inicialização ---
    pesos_p1 = [0.0] * (max_coef + 1)
    pesos_p2 = [0.0] * (max_coef + 1)
    start_game = 0

    if os.path.exists(FILE_PESOS_P1):
        print(f"Carregando pesos existentes de {FILE_PESOS_P1}...")
        pesos_p1 = list(np.load(FILE_PESOS_P1))
        pesos_p2 = list(np.load(FILE_PESOS_P2))
        print("Pesos P1 carregados (antes do treino):")
        print([round(p, 4) for p in pesos_p1])
        print("Pesos P2 carregados (antes do treino):")
        print([round(p, 4) for p in pesos_p2])
    else:
        print("Iniciando com pesos zerados.")

    # --- Inicializa Log de Winrates (LOG_FILE) ---
    header_log_winrates = [
        "partida_num", "g_por_s",
        "rand_winrate", "rand_w", "rand_l", "rand_d",
        "pos_winrate", "pos_w", "pos_l", "pos_d",
        "pesos_p1", "pesos_p2"
    ]
    log_winrates_existe = os.path.exists(LOG_FILE)
    if not log_winrates_existe:
        with open(LOG_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(header_log_winrates)
        print(f"Novo arquivo de log (winrates) criado em: {LOG_FILE}")
    else:
        try:
            df = pd.read_csv(LOG_FILE)
            if not df.empty:
                start_game = df['partida_num'].iloc[-1]
                print(f"Histórico (winrates) encontrado. Continuando treino a partir da partida {start_game}")
            else:
                print(f"Arquivo de log (winrates) encontrado, mas está vazio.")
        except Exception as e:
            print(f"AVISO: Não foi possível ler o log {LOG_FILE}. Reiniciando contagem. Erro: {e}")
            start_game = 0

    # --- Inicializa Log de Pesos (PESOS_LOG_FILE) ---
    pesos_header = ["partida_num"]
    for i in range(max_coef + 1): pesos_header.append(f"p1_w{i}")
    for i in range(max_coef + 1): pesos_header.append(f"p2_w{i}")

    if not os.path.exists(PESOS_LOG_FILE):
        with open(PESOS_LOG_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(pesos_header)
        print(f"Novo arquivo de log (pesos) criado em: {PESOS_LOG_FILE}")
    else:
        print(f"Continuando log (pesos) em: {PESOS_LOG_FILE}")

    # --- Instancia os Jogadores ---

    # O Jogador 1 (P1) é o nosso "Agente" principal que sempre treina
    jogador_p1 = JogadorTabelaPosicoes(
        indice_coeficientes,
        pesos_p1, pesos_p2,
        alpha=ALPHA, epsilon=EPSILON_TREINO, gamma=GAMMA_TREINO,
        is_training=True
    )

    # O Jogador 2 (P2) é o "Oponente" que vamos escolher a cada jogo
    # Ele também treina (no caso do self-play)
    jogador_p2_treinador = JogadorTabelaPosicoes(
        indice_coeficientes,
        pesos_p1, pesos_p2,
        alpha=ALPHA, epsilon=EPSILON_TREINO, gamma=GAMMA_TREINO,
        is_training=True
    )

    # Oponentes "estáticos" (não aprendem)
    oponente_posicional = JogadorPosicional()
    oponente_aleatorio = JogadorAleatorio()


    print(f"Iniciando treino (Self-Play) de {start_game} até {NUM_PARTIDAS_TREINO} partidas...")
    print(f"Parâmetros: Alpha={ALPHA}, Epsilon={EPSILON_TREINO}, Gamma={GAMMA_TREINO}")

    start_time_total = time.time()
    games_since_last_eval = 0

    # --- Loop de Treino Principal ---
    for i in range(start_game, NUM_PARTIDAS_TREINO):

        p = random.random()

        if p < 0.7: # 70% das partidas são de Self-Play
            j1 = jogador_p1
            j2 = jogador_p2_treinador
        elif p < 0.9: # 20% das partidas vs Posicional (para não esquecer)
            j1 = jogador_p1
            j2 = oponente_posicional # P2 não aprende
        else: # 10% das partidas vs Aleatório (para manter o básico)
            j1 = jogador_p1
            j2 = oponente_aleatorio # P2 não aprende

        play_game(j1, j2)

        games_since_last_eval += 1

        # --- Bloco de Avaliação e Log ---
        if games_since_last_eval >= AVALIAR_RANDOM_A_CADA:
            games_since_last_eval = 0
            partida_atual = i + 1

            elapsed_total = time.time() - start_time_total
            games_per_sec = (partida_atual) / elapsed_total

            print(f"\n--- Partida {partida_atual}/{NUM_PARTIDAS_TREINO} ({games_per_sec:.0f} g/s) ---")

            # 1. Avalia contra Aleatório
            eval_rand = evaluate_vs_random(pesos_p1, pesos_p2, games=JOGOS_AVAL_RANDOM)
            winrate_rand = eval_rand["winrate"]
            print(f"Avaliação vs Aleatório: {winrate_rand*100:.1f}% de vitórias")
            print(f"  (W: {eval_rand['wins']}, L: {eval_rand['losses']}, D: {eval_rand['draws']})")

            # 2. Avalia contra Posicional (só se for a hora certa)
            eval_pos = {"winrate": -1, "wins": -1, "losses": -1, "draws": -1}
            if partida_atual % AVALIAR_POS_A_CADA_N_JOGOS == 0:
                print("--- AVALIAÇÃO ESPECIAL vs POSICIONAL ---")
                eval_pos = evaluate_vs_positional(pesos_p1, pesos_p2, games=JOGOS_AVAL_POS)
                winrate_pos = eval_pos["winrate"]
                print(f"Avaliação vs Posicional: {winrate_pos*100:.1f}% de vitórias")
                print(f"  (W: {eval_pos['wins']}, L: {eval_pos['losses']}, D: {eval_pos['draws']})")

            # 3. Salva pesos no Google Drive
            print(f"Salvando pesos em {FILE_PESOS_P1}...")
            np.save(FILE_PESOS_P1, pesos_p1)
            np.save(FILE_PESOS_P2, pesos_p2)

            pesos_atuais_p1_str = str([round(p, 4) for p in pesos_p1])
            pesos_atuais_p2_str = str([round(p, 4) for p in pesos_p2])
            print(f"Pesos P1 Atuais: {pesos_atuais_p1_str}")
            print(f"Pesos P2 Atuais: {pesos_atuais_p2_str}")

            # 4. Adiciona linha ao LOG CSV (Winrates)
            try:
                with open(LOG_FILE, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        partida_atual, round(games_per_sec, 1),
                        round(eval_rand["winrate"], 3), eval_rand["wins"], eval_rand["losses"], eval_rand["draws"],
                        round(eval_pos["winrate"], 3), eval_pos["wins"], eval_pos["losses"], eval_pos["draws"],
                        pesos_atuais_p1_str,
                        pesos_atuais_p2_str
                    ])
                print(f"Progresso (winrate) salvo em {LOG_FILE}")
            except Exception as e:
                print(f"ERRO AO SALVAR LOG (winrate): {e}")

            # 5. Adiciona linha ao LOG CSV (Pesos)
            try:
                with open(PESOS_LOG_FILE, 'a', newline='') as f:
                    writer = csv.writer(f)
                    row = [partida_atual] + [round(p, 4) for p in pesos_p1] + [round(p, 4) for p in pesos_p2]
                    writer.writerow(row)
                print(f"Progresso (pesos) salvo em {PESOS_LOG_FILE}")
            except Exception as e:
                print(f"ERRO AO SALVAR LOG (pesos): {e}")


    print("\n--- TREINAMENTO CONCLUÍDO ---")

    # --- Bloco de Plotagem (Winrates) ---
    if os.path.exists(LOG_FILE):
        print("Gerando gráfico final de WINRATES a partir do log...")
        try:
            df = pd.read_csv(LOG_FILE)

            df_rand = df[df['rand_winrate'] >= 0]
            df_pos = df[df['pos_winrate'] >= 0]

            plt.figure(figsize=(10, 6))

            plt.plot(df_rand['partida_num'], df_rand['rand_winrate'], label="Winrate vs Aleatório", marker='o')

            if not df_pos.empty:
                plt.plot(df_pos['partida_num'], df_pos['pos_winrate'], label="Winrate vs Posicional", marker='x', linestyle='--')

            plt.title(f"Progresso do Treino (TD-0, Alpha={ALPHA})")
            plt.xlabel("Partidas Treinadas")
            plt.ylabel("Winrate")
            plt.legend()
            plt.grid(True)
            plt.ylim(0, 1)

            plt.savefig(PLOT_FILE)
            print(f"Gráfico de winrates salvo em: {PLOT_FILE}")
            plt.show()

        except Exception as e:
            print(f"ERRO ao gerar gráfico de winrates: {e}")

    # --- Bloco de Plotagem (Pesos) ---
    if os.path.exists(PESOS_LOG_FILE):
        print("Gerando gráfico final de EVOLUÇÃO DOS PESOS (P1) a partir do log...")
        try:
            df_pesos = pd.read_csv(PESOS_LOG_FILE)

            plt.figure(figsize=(12, 7))

            for i in range(max_coef + 1):
                col_name = f'p1_w{i}'
                label = f'P1 Peso {i}'
                if i == 0: label = f'P1 Peso {i} (Cantos)'
                if i == 1: label = f'P1 Peso {i} (Casas-C)'
                if i == 4: label = f'P1 Peso {i} (Casas-X)'

                plt.plot(df_pesos['partida_num'], df_pesos[col_name], label=label)

            plt.title("Evolução dos Pesos (Jogador 1)")
            plt.xlabel("Partidas Treinadas")
            plt.ylabel("Valor do Peso")
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.grid(True)
            plt.tight_layout()

            plt.savefig(PLOT_PESOS_FILE)
            print(f"Gráfico de pesos salvo em: {PLOT_PESOS_FILE}")
            plt.show() # Mostra o gráfico no Colab

        except Exception as e:
            print(f"ERRO ao gerar gráfico de pesos: {e}")

In [ ]:
# =================================================================================
# AVALIADOR DE HISTÓRICO COMPLETO (TD-0 vs BASLINES com LIVRO DE ABERTURAS)
# =================================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import random

# 1. Configuração e Drive
try:
    from google.colab import drive
    print("Montando Google Drive...")
    drive.mount('/content/drive')
    SAVE_DIR = "/content/drive/MyDrive/OthelloTreinoTD0"
except ImportError:
    print("Ambiente local detectado.")
    SAVE_DIR = "OthelloTreinoTD0_local"

LOG_FILE = os.path.join(SAVE_DIR, "treino_log.csv")

# =================================================================================
# 2. CLASSES DO JOGO E JOGADORES
# =================================================================================

class Othello:
    direcoes = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
    def __init__(self, outro=None):
        if outro:
            self._tab = np.array(outro._tab, dtype=np.int8)
            self._jogador = outro._jogador
            self._terminou = outro._terminou
        else:
            self._tab = np.zeros((8,8), dtype=np.int8)
            self._tab[3,3], self._tab[4,4] = -1, -1
            self._tab[3,4], self._tab[4,3] = 1, 1
            self._jogador = 1
            self._terminou = False
        self._cache_legal = None

    def tabuleiro(self): return self._tab
    def jogador_atual(self): return self._jogador
    def placar(self): return (np.sum(self._tab==1), np.sum(self._tab==-1))
    def posicao(self, i, j): return self._tab[i,j]

    def _capturas(self, i, j, player):
        if self._tab[i,j] != 0: return []
        caps = []
        for di, dj in self.direcoes:
            ci, cj = i+di, j+dj
            temp = []
            while 0<=ci<8 and 0<=cj<8 and self._tab[ci,cj] == -player:
                temp.append((ci,cj))
                ci, cj = ci+di, cj+dj
            if 0<=ci<8 and 0<=cj<8 and self._tab[ci,cj] == player:
                caps.extend(temp)
        return caps

    def jogadas_legais(self):
        if self._terminou: return []
        if self._cache_legal: return list(self._cache_legal)
        l = []
        for i in range(8):
            for j in range(8):
                if self._capturas(i,j,self._jogador): l.append((i,j))
        self._cache_legal = l
        return l

    def joga(self, jogada):
        novo = Othello(self)
        caps = novo._capturas(jogada[0], jogada[1], novo._jogador)
        novo._tab[jogada] = novo._jogador
        for c in caps: novo._tab[c] = novo._jogador
        novo._jogador *= -1
        if not novo.jogadas_legais():
            novo._jogador *= -1
            if not novo.jogadas_legais(): novo._terminou = True
        return novo

    def terminou(self): return self._terminou

# Mapa de Coeficientes
indice_coeficientes = [
  [0, 1, 2, 3, 3, 2, 1, 0], [1, 4, 5, 6, 6, 5, 4, 1],
  [2, 5, 7, 8, 8, 7, 5, 2], [3, 6, 8, 9, 9, 8, 6, 3],
  [3, 6, 8, 9, 9, 8, 6, 3], [2, 5, 7, 8, 8, 7, 5, 2],
  [1, 4, 5, 6, 6, 5, 4, 1], [0, 1, 2, 3, 3, 2, 1, 0]]
max_coef = 9

# --- JOGADOR ALEATÓRIO ---
class JogadorAleatorio:
    def escolhe_jogada(self, jogo):
        l = jogo.jogadas_legais()
        return l[np.random.randint(len(l))] if l else None
    def nova_partida(self, *args): pass
    def informa_propria_jogada(self, *args): pass
    def informa_jogada_oponente(self, *args): pass
    def informa_fim(self, *args): pass

# --- JOGADOR POSICIONAL ---
class JogadorPosicional:
    def __init__(self):
        self.w = {"corner": 100, "x_pen": 80, "c_pen": 40, "mob": 5, "edge": 2, "disc": 0.2}
        self.corners = [(0,0),(0,7),(7,0),(7,7)]
        self.x_sq = {(0,0):(1,1), (0,7):(1,6), (7,0):(6,1), (7,7):(6,6)}
        self.c_sq = {(0,0):[(0,1),(1,0)], (0,7):[(0,6),(1,7)], (7,0):[(6,0),(7,1)], (7,7):[(6,7),(7,6)]}

    def nova_partida(self, j, p, o=None): self.p = p

    def escolhe_jogada(self, jogo):
        best, move = -1e9, None
        for a in jogo.jogadas_legais():
            s = self._eval(jogo.joga(a), self.p)
            if s > best: best, move = s, a
        return move

    def _eval(self, jogo, p):
        t = jogo.tabuleiro()
        sc = 0
        for c in self.corners:
            if t[c] == p: sc += self.w['corner']
            elif t[c] == -p: sc -= self.w['corner']
        for c in self.corners:
            if t[c] == 0:
                if t[self.x_sq[c]] == p: sc -= self.w['x_pen']
                for ca in self.c_sq[c]:
                    if t[ca] == p: sc -= self.w['c_pen']
        j_temp = Othello(jogo); j_temp._jogador = p; j_temp._cache_legal = None
        m_my = len(j_temp.jogadas_legais())
        j_temp._jogador = -p; j_temp._cache_legal = None
        m_op = len(j_temp.jogadas_legais())
        sc += (m_my - m_op) * self.w['mob']
        return sc

    def informa_propria_jogada(self, *args): pass
    def informa_jogada_oponente(self, *args): pass
    def informa_fim(self, *args): pass

# --- JOGADOR TD(0) (Somente Leitura) ---
class JogadorTD0:
    def __init__(self, w1, w2):
        self.w1, self.w2 = w1, w2
        self.idx = indice_coeficientes

    def nova_partida(self, jogo, jogador, op=None):
        self.p = jogador
        self.my_w = self.w1 if self.p == 1 else self.w2
        self.op_w = self.w2 if self.p == 1 else self.w1

    def _feat(self, jogo, persp):
        f = [0.0]*10
        for r in range(8):
            for c in range(8):
                f[self.idx[r][c]] += jogo.posicao(r,c) * persp
        return f

    def _val(self, feat, w):
        lin = sum(f*w_i for f, w_i in zip(feat, w))
        return 2 / (1 + np.exp(-2 * lin)) - 1

    def escolhe_jogada(self, jogo):
        l = jogo.jogadas_legais()
        # Modo Avaliação: SEMPRE escolhe a melhor (Greedy)
        vals = []
        for a in l:
            nxt = jogo.joga(a)
            feat_op = self._feat(nxt, -self.p)
            v_op = self._val(feat_op, self.op_w)
            vals.append(-v_op)
        return l[np.argmax(vals)]

    def informa_propria_jogada(self, *args): pass
    def informa_jogada_oponente(self, *args): pass
    def informa_fim(self, *args): pass

# =================================================================================
# 3. FUNÇÕES DE AVALIAÇÃO COM LIVRO
# =================================================================================

def create_opening_book(n=200, depth=6):
    print(f"Gerando Livro de Aberturas ({n} posições, profundidade {depth})...")
    book = []
    jr = JogadorAleatorio()
    for _ in range(n):
        g = Othello()
        try:
            for _ in range(depth):
                if g.terminou(): break
                g = g.joga(jr.escolhe_jogada(g))
            book.append(g)
        except: pass
    return book

def play(j1, j2, start_board=None):
    g = Othello(start_board) if start_board else Othello()
    p_inicial = g.jogador_atual()


    if p_inicial == 1:
        j1.nova_partida(g, 1); j2.nova_partida(g, -1)
    else:
        j2.nova_partida(g, 1); j1.nova_partida(g, -1)

    while not g.terminou():
        p = g.jogador_atual()
        if p == 1: g = g.joga(j1.escolhe_jogada(g) if p_inicial==1 else j2.escolhe_jogada(g))
        else:      g = g.joga(j2.escolhe_jogada(g) if p_inicial==1 else j1.escolhe_jogada(g))

    p1, p2 = g.placar()
    res = 1 if p1 > p2 else (-1 if p2 > p1 else 0)
    return res

def evaluate_batch(w1, w2, opponent_cls, book):
    wins, total = 0, 0
    for start_node in book:
        j_td = JogadorTD0(w1, w2)
        j_op = opponent_cls()

        # Jogo 1: TD joga como P1 (Pretas)
        res = play(j_td, j_op, start_board=start_node)
        if res == 1: wins += 1
        total += 1

        # Jogo 2: TD joga como P2 (Brancas)
        j_td = JogadorTD0(w1, w2)
        j_op = opponent_cls()
        res = play(j_op, j_td, start_board=start_node)
        if res == -1: wins += 1
        total += 1

    return wins / total

# Parser de pesos do CSV
def parse_weights(s):
    matches = re.findall(r"(-?\d+\.\d+)", str(s))
    if len(matches) == 10: return [float(m) for m in matches]
    return [0.0]*10 # Fallback

# =================================================================================
# 4. EXECUÇÃO PRINCIPAL
# =================================================================================

if not os.path.exists(LOG_FILE):
    print(f"ERRO: Arquivo de log não encontrado em {LOG_FILE}")
else:
    print(f"Lendo histórico de {LOG_FILE}...")
    df = pd.read_csv(LOG_FILE)

    # Filtra apenas as linhas válidas
    df = df.dropna(subset=['pesos_p1', 'pesos_p2'])

    print(f"Encontrados {len(df)} checkpoints. Iniciando re-avaliação robusta...")

    # Cria um livro de aberturas FIXO para ser justo em todos os checkpoints
    # 50 aberturas * 2 lados = 100 jogos por avaliação
    BOOK = create_opening_book(n=200, depth=4)

    results = []

    start_time = time.time()

    for idx, row in df.iterrows():
        partida = int(row['partida_num'])
        w1 = parse_weights(row['pesos_p1'])
        w2 = parse_weights(row['pesos_p2'])

        if sum(w1) == 0 and sum(w2) == 0: continue

        print(f"Avaliando checkpoint {partida}...", end=" ")

        # Avalia vs Random
        wr_rand = evaluate_batch(w1, w2, JogadorAleatorio, BOOK)

        # Avalia vs Posicional
        wr_pos = evaluate_batch(w1, w2, JogadorPosicional, BOOK)

        print(f"Rand: {wr_rand:.1%}, Pos: {wr_pos:.1%}")

        results.append({
            'partida': partida,
            'winrate_random_book': wr_rand,
            'winrate_posicional_book': wr_pos,
            'p1_canto': w1[0],
            'p1_casa_c': w1[1],
            'p1_casa_x': w1[4]
        })

    # Cria DataFrame dos resultados
    df_res = pd.DataFrame(results)

    # Plota os Resultados
    plt.figure(figsize=(12, 6))
    plt.plot(df_res['partida'], df_res['winrate_random_book'], label='vs Aleatório (Livro)', marker='o')
    plt.plot(df_res['partida'], df_res['winrate_posicional_book'], label='vs Posicional (Livro)', marker='x', linestyle='--')
    plt.title("Evolução Real da Performance (Avaliação Robusta)")
    plt.xlabel("Partidas Treinadas")
    plt.ylabel("Winrate")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.05)

    plt.savefig('avaliacao_robusta_final.png')
    plt.show()

    print("\nAnálise concluída! Gráfico salvo como 'avaliacao_robusta_final.png'.")
    print("Mostrando tabela final:")
    print(df_res[['partida', 'winrate_random_book', 'winrate_posicional_book']].tail(10))